In [ ]:
pip install SpeechRecognition transformers torch

In [ ]:
pip install tf-keras

In [ ]:
import speech_recognition as sr
from transformers import pipeline

# Function to capture audio input from user
def capture_audio_until_stop():
    recognizer = sr.Recognizer()
    full_text = []
    with sr.Microphone() as source:
        while True:
            print("Please speak (say 'stop' to end)...")
            audio = recognizer.listen(source)

            try:
                text = recognizer.recognize_google(audio)
                print(f"You said: {text}")  # Debugging: check what was recognized
                if "stop" in text.lower():
                    print("Stopping the recording.")
                    text = text.lower().replace("stop", "").strip()  # Remove 'stop' and trim whitespace
                    full_text.append(text)
                    break
                full_text.append(text)
            except sr.UnknownValueError:
                print("Sorry, I could not understand the audio.")
            except sr.RequestError:
                print("Request error; check your network connection.")
    
    final_text = " ".join(full_text).strip()  # Join all parts into one string

    print(f"\nFinal captured text after removing 'stop': {final_text}")  # Debugging statement
    return final_text


from transformers import pipeline
import torch

# Check if a GPU is available and set the device accordingly
device = 0 if torch.cuda.is_available() else -1


# Function to analyze emotion from text
def analyze_emotion(text):
    emotion_analyzer = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=True)
    emotions = emotion_analyzer(text)
    return emotions

if __name__ == "__main__":
    # Capture audio from user
    user_input = capture_audio_until_stop()
    
    if user_input:
        print(f"\nFinal captured text: {user_input}")  # Debugging statement
        
        # Analyze the emotion of the captured text
        emotions = analyze_emotion(user_input)
        print("\nEmotion analysis results:")  # Debugging statement
        for emotion in emotions[0]:
            print(f"{emotion['label']}: {emotion['score']:.4f}")

        # Determine top emotion
        top_emotion = max(emotions[0], key=lambda x: x['score'])['label']
        print(f"\nTop detected emotion: {top_emotion}")  # Debugging statement
        
        # Example: Respond based on detected emotion
        if top_emotion == "joy":
            response = "I'm glad you're feeling happy. Let's keep that positivity going!"
        elif top_emotion == "sadness":
            response = "I'm sorry you're feeling sad. I'm here to listen."
        elif top_emotion == "anger":
            response = "It seems like you're feeling angry. Let's talk about what's on your mind."
        elif top_emotion == "fear":
            response = "It sounds like something is worrying you. How can I help?"
        else:
            response = "I'm here for you, no matter what you're feeling."

        print(f"\nBot response: {response}")
    else:
        print("No valid input was captured.")
